Hello dis is test

In [2]:
from Inner_Speech_Dataset.Python_Processing.Data_extractions import  Extract_data_from_subject

datatype = "EEG"
root_dir = "../dataset"
subjectNr = 1

data, label = Extract_data_from_subject(root_dir, subjectNr, datatype)

In [3]:
print(type(data[10])) # [39869     3     0     1]  I think it means this:  [unknown     class?     contaminated?     subjectNr]
print(data.shape)
print(data[0].shape)

# 500 trials

<class 'numpy.ndarray'>
(500, 128, 1153)
(128, 1153)


In [4]:
#!ls -la .. # ! to write bash commands

In [49]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

class EEGNet(nn.Module):
    def __init__(self):
        super(EEGNet, self).__init__()
        self.f1 = 16
        self.d = 4
        self.f2 = self.d*self.f1

        # Block 1
        self.conv1 = nn.Conv2d(in_channels=1, out_channels = self.f1, kernel_size = (1, 128), padding = 0, bias = False) # 128 = half sampling rate, reduces to 2Hz, 16 is probably F1, the number of filters (output channels)
        self.batchnorm1 = nn.BatchNorm2d(self.f1, False)

        self.conv2 = nn.Conv2d(in_channels = self.f1, out_channels = self.d, kernel_size = (128, 1), padding = 0, bias = False) # row, column] 128 = number of sensors for the depth wise convolution
        # I'm missing a maxnorm regilarization here.
        # Add dropout in the forward func.
        self.batchnorm2 = nn.BatchNorm2d(self.d, False) # Not sure this is the right number of features.
        self.elu = nn.ELU()
        self.avgPool1 = nn.AvgPool2d((1,4)) # Reduces sampling rate to 64hz

        # Block 2

        # Separable convolution
        self.depthwise = nn.Conv2d(4, 4, kernel_size = (1,32), padding=1, groups=4, bias = False) # Captures 500ms of data at sampling rate 64Hz (32 Hz is 500ms of that)
        self.pointwise = nn.Conv2d(4, self.f2, kernel_size = 1) # This results in f2 channels of the output here, not sure if it's meant to be like this. But I proceede to run a test

        self.batchnorm3 = nn.BatchNorm2d(self.f2, False)

        self.avgPool2 = nn.AvgPool2d((1,8))

        self.classifier = nn.Linear(5376,4, bias = False)
        self.softmax = nn.Softmax()


    def forward(self, x, dropout = 0.5): # Dropout 0.5 dropout for within-subject and 0.25 for cross-subject
        # Block 1
        res = self.conv1(x)
        res = self.batchnorm1(res)
        res = self.conv2(res)
        res = self.batchnorm2(res)
        # res = F.dropout(0.25)
        res = self.elu(res)
        res = self.avgPool1(res)
        res = F.dropout(res, dropout)
        res = self.depthwise(res)
        res = self.pointwise(res)
        res = self.batchnorm3(res)
        res = self.elu(res)
        res = self.avgPool2(res)
        res = F.dropout(res, dropout)
        res = torch.flatten(res)
        res = self.classifier(res)
        res = self.softmax(res)
        return res


In [57]:
network = EEGNet().float()
d = torch.tensor(data[1])
print(d.size())
d = torch.unsqueeze(torch.unsqueeze(d,0),0).float() # Unsqueeze adds a wrapper dimension of 1s
print(d.size())
r = network.forward(d)
#print(r.size()) # [1,4,1,256] [samples, channels, height, width]
# [1,64,3,28] right before flatten leads to 64*3*28 = 5376 dim
print(r)

print(torch.max(r,0))


torch.Size([128, 1153])
torch.Size([1, 1, 128, 1153])
tensor([0.2746, 0.2421, 0.2711, 0.2122], grad_fn=<SoftmaxBackward0>)
torch.return_types.max(
values=tensor(0.2746, grad_fn=<MaxBackward0>),
indices=tensor(0))


/tmp/ipykernel_67299/3518565424.py:56: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  res = self.softmax(res)
